### Set up

#### 1. Set  up  accounts and role

In [1]:
import sagemaker
import boto3
from uuid import uuid4

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)
max_runs=1

#### 2. Setup image and instance type

In [2]:
pytorch_custom_image_name="bertreverse:gpu-202201301844"
instance_type = "ml.p3.2xlarge"
instance_type_gpu_map = {"ml.p3.8xlarge":4, "ml.p3.2xlarge": 1, "ml.p3.16xlarge":8}

In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [4]:
bucket = "aegovan-data"

In [5]:
s3_input_vocab ="s3://{}/embeddings/bert_base_cased/".format(bucket)

s3_input_data = "s3://{}/mnli_dataset/val/multinli_1.0_dev_matched.jsonl".format(bucket)

s3_model_path= "s3://{}/mnli_sagemakerresults/mnli-reverse-lang-bert-accuracy-2022-01-23-21-29-34-327/output/".format(bucket)
s3_code_path= "s3://{}/mnli_bert_code".format(bucket)
s3_output_predictions = "s3://{}/mnli_predictions".format(bucket)

### Start prediction

In [6]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit 85ec10ff4f4f5eae6947caa0ebe85b99b6bbe996
    Bug Fix - Model not found


In [7]:
# set True if you need spot instance
instance_count=1
use_spot = False
train_max_run_secs =   5 *24 * 60 * 60
spot_wait_sec =  60 * 60 * 5
max_wait_time_secs = train_max_run_secs +  spot_wait_sec

if not use_spot:
    max_wait_time_secs = None
    
# During local mode, no spot.., use smaller dataset
if instance_type == 'local':
    use_spot = False
    max_wait_time_secs = 0
    wait = True
    instance_count=1
    # Use smaller dataset to run locally
    inputs = inputs_sample

In [8]:
s3_data_type="S3Prefix"

In [9]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=instance_count,
                                       max_runtime_in_seconds= 5 * 24 * 60 * 60,
                                       volume_size_in_gb = 250,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="bert-reverse-inference"
                                       )


sm_local_input_models = "/opt/ml/processing/input/data/models"
sm_local_input_data = "/opt/ml/processing/input/data/jsonlines"
sm_local_input_vocab = "/opt/ml/processing/input/data/vocab"


sm_local_output = "/opt/ml/processing/output"



script_processor.run(
        code='../src/inference/reverse_lang_mnli_batch_predict.py',

        arguments=[
            sm_local_input_data,
            sm_local_input_models,
            sm_local_output,
            "--tokenisor_data_dir", sm_local_input_vocab,
            "--batch", "32",
            "vocab_file", f"{sm_local_input_vocab}/vocab.txt" 
        ],

        inputs=[
                ProcessingInput(
                    source=s3_input_data,
                    s3_data_type = s3_data_type,
                    destination=sm_local_input_data,
                    s3_data_distribution_type="FullyReplicated"),

                ProcessingInput(
                        source=s3_model_path,
                        destination=sm_local_input_models,
                        s3_data_distribution_type="FullyReplicated"),

                ProcessingInput(
                        source=s3_input_vocab,
                        destination=sm_local_input_vocab,
                        s3_data_distribution_type="FullyReplicated")
            ],


        outputs=[ProcessingOutput(
                source=sm_local_output, 
                destination=s3_output_predictions,
                output_name='predictions')]
    )






Job Name:  bert-reverse-inference-2022-01-30-18-52-52-271
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/mnli_dataset/val/multinli_1.0_dev_matched.jsonl', 'LocalPath': '/opt/ml/processing/input/data/jsonlines', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/mnli_sagemakerresults/mnli-reverse-lang-bert-accuracy-2022-01-23-21-29-34-327/output/', 'LocalPath': '/opt/ml/processing/input/data/models', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/embeddings/bert_base_cased/', 'LocalPath': '/opt/ml/processing/input/data/vocab', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 